<h1>Generate Answers from MedVet with API</h2>
In this notebook we generate the answers to the described cases with the API of Medvet.
The creation will be done one time with a combined answer of an image from LLaVA-Med and the LlaMA 2 RAG system and one time with a separate output.

Specifications:

---------------------------
<p>Used model in MedVet: LlaMA 2 7b-chat with 8bit quantization as .gguf format </p>
<p>used prompt template: "Casesolver"</p>

In [1]:
import requests
import base64
from PIL import Image
from io import BytesIO
import json

In [2]:
ip_address_llava = "" #Have to be updated after start of EC2 instance
ip_address_medvet = "127.0.0.1:8000"

url = f"http://{ip_address_medvet}/generate"
clear_url_llava = f"http://{ip_address_llava}/clearchat"
clear_url_llama = f"http://{ip_address_medvet}/clearchat"

In [4]:
def clearChat():
    try:
        if ip_address_llava != "":
            requests.post(clear_url_llava)
        if ip_address_medvet != "":
            requests.post(clear_url_llama)
    except Exception as e:
        print( f"Failed to connect to AWS:{e}")

In [24]:
def getAnswerMedVet(case,image_name,combined:bool,chaintype):
    output = ""
    data = {
                "prompt": [{"role":"assistant","content": "How may I assist you?"},{"role":"user","content": f"{case}"}],
                "agent_id": "validation_02",
                "ip_address_llava": "",
                "img": "",
                "chaintype":chaintype,
                "display_combined": combined,
                "mode_assistant": "simple_case" ,
                "llm_template_name": "llama2_plain" if chaintype == "plain" else "llama2_RAG",
                "use_rag": 1,
                "temperature": 0.7,
                "max_new_tokens": 1024
            }
    headers = {'Content-type': 'application/json', 'Accept': 'application/json'}
    try:
        response = requests.put(url, data=json.dumps(data), headers=headers)
        print(f"Response: {response}")
        response = response.json()
        output = response

    except Exception as e:
        output = f"Failed to connect to AWS:{e}"
    return output

<h1>Combined Answers</h1>

In [27]:
import csv

lines_newCSV = []
with open('Evaluation Cases Jan.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        answer = getAnswerMedVet(row['Case'],row['Image'],0,"rag")
        answer2 = getAnswerMedVet(row['Case'],row['Image'],0,"plain")
        lines_newCSV.append({
            "Number":line_count,
            "Case": row['Case'],
            "Image": row['Image'],
            "Solution": row['Solution'],
            "Answer LLaMA 2 finetuned plain": answer2['answer_llama']['answer'],
            "Answer LLaMA 2 finetuned RAG": answer['answer_llama']['answer'],
            "Sources RAG": answer['answer_llama']['sources']
            })
        clearChat()
        line_count +=1


Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>
Response: <Response [200]>


In [29]:
with open('Results/EvaluationFInetunedLLaMA2_3.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Case", 'Image','Solution',"Sources RAG","Answer LLaMA 2 finetuned plain",'Answer LLaMA 2 finetuned RAG']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)

<h1>Separate</h1>

In [ ]:
import csv

lines_newCSV = []
with open('Evaluation Cases.csv', mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    for row in csv_reader:
        answer = getAnswerMedVet(row['Case'],row['Image'],0)
        answer_llama = ""
        answer_llava = ""
        if "answer_llama" in answer:
            answer_llama = answer['answer_llama']['result']
        if "answer_llava" in answer:
            answer_llava = answer['answer_llava']['result']

        lines_newCSV.append({
            "Number":line_count,
            "Case": row['Case'],
            "Image": row['Image'],
            "Solution": row['Solution'],
            "Source": row['Source'],
            "Answer MedVet LlaMA": answer_llama,
            "Answer MedVet LlaVA": answer_llava,
            "Answer ChatGPT": row['Answer ChatGPT']
            })
        clearChat()
        line_count +=1


In [1]:
with open('Results/MedVetAnswers_separated.csv', 'w', newline='') as csvfile:
    fieldnames = ['Number', "Case", 'Image','Solution','Source',"Answer MedVet LlaMA","Answer MedVet LlaVA","Answer ChatGPT"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(lines_newCSV)

NameError: name 'csv' is not defined